<a href="https://colab.research.google.com/github/devi1enok/deeplearning/blob/main/20233574_%EB%B0%A9%EC%97%98%EB%A0%88%EC%98%A4%EB%85%B8%EB%9D%BC_p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


practice 1-1,1-2

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Split the training dataset into training and validation (4:1 ratio)
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
train_data, valid_data = random_split(train_dataset, [train_size, valid_size])

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define the MLP Model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)  # First hidden layer with 128 neurons
        self.fc2 = nn.Linear(128, 512)      # Second hidden layer with 512 neurons
        self.fc3 = nn.Linear(512, 10)       # Output layer with 10 neurons (10 classes for digits)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)      # Dropout to reduce overfitting

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.dropout(self.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

model = MLP()
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-5)

class EarlyStopping:
    def __init__(self, patience=5, delta=0.001):
        self.patience = patience
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.delta = delta

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

def train_model(model, train_loader, valid_loader, criterion, optimizer, patience=5, n_epochs=50):
    early_stopping = EarlyStopping(patience=patience)
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        for data, target in train_loader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation step
        valid_loss = 0.0
        model.eval()
        with torch.no_grad():
            for data, target in valid_loader:
                outputs = model(data)
                loss = criterion(outputs, target)
                valid_loss += loss.item()

        # Print losses for each epoch
        print(f"Epoch {epoch+1}, Training Loss: {running_loss/len(train_loader)}, Validation Loss: {valid_loss/len(valid_loader)}")

        # Check if early stopping should be triggered
        early_stopping(valid_loss/len(valid_loader))
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break


train_model(model, train_loader, valid_loader, criterion, optimizer, patience=5, n_epochs=50)

def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    print(f'Accuracy of the model on the 10000 test images: {100 * correct / total}%')

test_model(model, test_loader)

Epoch 1, Training Loss: 1.0668238195180892, Validation Loss: 0.457376240732822
Epoch 2, Training Loss: 0.43481359507640205, Validation Loss: 0.3627378516136966
Epoch 3, Training Loss: 0.36342609065771103, Validation Loss: 0.31871882037437976
Epoch 4, Training Loss: 0.3252552382051945, Validation Loss: 0.2948176242965967
Epoch 5, Training Loss: 0.2995786269803842, Validation Loss: 0.2752205325845391
Epoch 6, Training Loss: 0.27673418053984644, Validation Loss: 0.25581884376229125
Epoch 7, Training Loss: 0.2560630637357632, Validation Loss: 0.24425585573895814
Epoch 8, Training Loss: 0.23907936177154382, Validation Loss: 0.22534744047183305
Epoch 9, Training Loss: 0.2255825720926126, Validation Loss: 0.21299094681013772
Epoch 10, Training Loss: 0.21154054437081019, Validation Loss: 0.2025374865674592
Epoch 11, Training Loss: 0.1994259590258201, Validation Loss: 0.19004350608373258
Epoch 12, Training Loss: 0.18885509766638278, Validation Loss: 0.1816641545478017
Epoch 13, Training Loss: 0

practice 2-1

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# Split training data into training and validation sets (4:1 ratio)
train_size = int(0.8 * len(train_dataset))
valid_size = len(train_dataset) - train_size
train_data, valid_data = random_split(train_dataset, [train_size, valid_size])

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Define MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(28 * 28, 128)
        self.fc2 = nn.Linear(128, 512)
        self.fc3 = nn.Linear(512, 10)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.view(-1, 28 * 28)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate model and loss function
model = MLP()
criterion = nn.CrossEntropyLoss()

# Early Stopping Class
class EarlyStopping:
    def __init__(self, patience=5, delta=0.001):
        self.patience = patience
        self.counter = 0
        self.best_loss = None
        self.early_stop = False
        self.delta = delta

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0

# Training function with Early Stopping
def train_model(model, train_loader, valid_loader, criterion, optimizer, patience=5, n_epochs=50):
    early_stopping = EarlyStopping(patience=patience)
    for epoch in range(n_epochs):
        model.train()
        running_loss = 0.0
        correct = 0
        total = 0
        for data, target in train_loader:
            optimizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

        train_accuracy = 100 * correct / total
        valid_loss, valid_accuracy = evaluate_model(model, valid_loader, criterion)

        print(f'Epoch {epoch+1}/{n_epochs}, '
              f'Training Loss: {running_loss/len(train_loader):.4f}, '
              f'Training Accuracy: {train_accuracy:.2f}%, '
              f'Validation Loss: {valid_loss:.4f}, '
              f'Validation Accuracy: {valid_accuracy:.2f}%')

        early_stopping(valid_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered")
            break

def evaluate_model(model, data_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    loss = 0.0
    with torch.no_grad():
        for data, target in data_loader:
            outputs = model(data)
            loss += criterion(outputs, target).item()
            _, predicted = torch.max(outputs, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
    accuracy = 100 * correct / total
    return loss / len(data_loader), accuracy

def test_model(model, test_loader):
    test_loss, test_accuracy = evaluate_model(model, test_loader, criterion)
    print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')

# Training and Testing with L2 Regularization (weight_decay)
optimizer_with_l2 = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-5)  # L2 regularization
print("Training with L2 regularization:")
train_model(model, train_loader, valid_loader, criterion, optimizer_with_l2, patience=5, n_epochs=50)

# Test the model with L2 regularization
print("Testing with L2 regularization:")
test_model(model, test_loader)

# Training and Testing without L2 Regularization
model = MLP()  # Reset the model
optimizer_without_l2 = optim.SGD(model.parameters(), lr=0.01, weight_decay=0)  # No L2 regularization
print("\nTraining without L2 regularization:")
train_model(model, train_loader, valid_loader, criterion, optimizer_without_l2, patience=5, n_epochs=50)

# Test the model without L2 regularization
print("Testing without L2 regularization:")
test_model(model, test_loader)


Training with L2 regularization:
Epoch 1/50, Training Loss: 1.0034, Training Accuracy: 73.89%, Validation Loss: 0.4523, Validation Accuracy: 87.22%
Epoch 2/50, Training Loss: 0.3790, Training Accuracy: 89.08%, Validation Loss: 0.3493, Validation Accuracy: 89.98%
Epoch 3/50, Training Loss: 0.3227, Training Accuracy: 90.60%, Validation Loss: 0.3108, Validation Accuracy: 90.96%
Epoch 4/50, Training Loss: 0.2915, Training Accuracy: 91.43%, Validation Loss: 0.2881, Validation Accuracy: 91.42%
Epoch 5/50, Training Loss: 0.2673, Training Accuracy: 92.15%, Validation Loss: 0.2765, Validation Accuracy: 91.98%
Epoch 6/50, Training Loss: 0.2465, Training Accuracy: 92.85%, Validation Loss: 0.2507, Validation Accuracy: 92.55%
Epoch 7/50, Training Loss: 0.2292, Training Accuracy: 93.27%, Validation Loss: 0.2380, Validation Accuracy: 93.04%
Epoch 8/50, Training Loss: 0.2119, Training Accuracy: 93.89%, Validation Loss: 0.2248, Validation Accuracy: 93.33%
Epoch 9/50, Training Loss: 0.1979, Training Acc